In [ ]:
import pandas as pd

In [2]:
customers=pd.read_csv("customers.csv")
products=pd.read_csv("products.csv")
orders=pd.read_csv("orders.csv")
order_items=pd.read_csv("order_items.csv")
returns=pd.read_csv("returns.csv")

# EXPLORATION

In [ ]:
# customers.head(3)
# customers.info()
# products.head(3)
# products.info()
orders.head()
# orders.info()
# order_items.head()

,order_id,order_date,customer_id
0,1,2024-01-17,1326
1,2,2024-01-29,1136
2,3,2024-01-17,1560
3,4,2024-01-15,3176
4,5,2024-01-03,957


# MOTIVE 
Orders Same ? 

In [16]:
# Create Month column in Orders
orders['Orders_Month']=pd.to_datetime(orders['order_date']).dt.month

In [17]:
orders.head()

,order_id,order_date,customer_id,Orders_Month
0,1,2024-01-17,1326,1
1,2,2024-01-29,1136,1
2,3,2024-01-17,1560,1
3,4,2024-01-15,3176,1
4,5,2024-01-03,957,1


In [21]:
# Monthly Orders Count 
monthly_orders = orders.groupby('Orders_Month')['order_id'].nunique().reset_index(name="total_Orders")
monthly_orders

,Orders_Month,total_Orders
0,1,800
1,2,800
2,3,800
3,4,800
4,5,800
5,6,800


# Revenue Trends

In [23]:
order_items.head()

,order_id,product_id,quantity,selling_price,discount_amount
0,1,3,1,12000,300
1,2,2,1,48000,300
2,3,3,1,12000,300
3,4,3,1,12000,300
4,5,1,1,10000,300


In [24]:
orders.head()

,order_id,order_date,customer_id,Orders_Month
0,1,2024-01-17,1326,1
1,2,2024-01-29,1136,1
2,3,2024-01-17,1560,1
3,4,2024-01-15,3176,1
4,5,2024-01-03,957,1


In [30]:
sales= order_items.merge(orders[['order_id','order_date']],on='order_id',how='left')
sales['order_month']=pd.to_datetime(sales['order_date']).dt.month

In [31]:
sales.head()

,order_id,product_id,quantity,selling_price,discount_amount,order_date,order_month
0,1,3,1,12000,300,2024-01-17,1
1,2,2,1,48000,300,2024-01-29,1
2,3,3,1,12000,300,2024-01-17,1
3,4,3,1,12000,300,2024-01-15,1
4,5,1,1,10000,300,2024-01-03,1


In [32]:
sales['revenue']=(sales['quantity']*sales['selling_price']-sales['discount_amount'])

In [34]:
#  Monthly Sales Revenue 
monthly_revenue = sales.groupby('order_month')['revenue'].sum().reset_index()
monthly_revenue

,order_month,revenue
0,1,19618000
1,2,19104000
2,3,18658000
3,4,19160000
4,5,18070000
5,6,18550000


In [37]:
# AVERAGE ORDER VALUE 

aov= sales.groupby('order_month').apply(lambda x: (x['revenue'].sum() / x['order_id'].nunique())).reset_index(name='aov')
aov

,order_month,aov
0,1,24522.5
1,2,23880.0
2,3,23322.5
3,4,23950.0
4,5,22587.5
5,6,23187.5


In [39]:
# monthly discount Trend
monthly_discount = sales.groupby('order_month')['discount_amount'].sum().reset_index()
monthly_discount

,order_month,discount_amount
0,1,240000
1,2,480000
2,3,720000
3,4,960000
4,5,1200000
5,6,1440000


In [40]:
# Returns Months

returns['return_month'] = pd.to_datetime(returns['return_date']).dt.to_period('M')

In [41]:
# return_monthly
returns_monthly = returns.groupby('return_month')['refund_amount'].sum().reset_index()
returns_monthly

,return_month,refund_amount
0,2024-05,3616674
1,2024-06,2771332


In [46]:
# Total Revenue - Total Refund Amount
total_revenue = monthly_revenue['revenue'].sum()
total_refund = returns_monthly['refund_amount'].sum()
net_revenue = total_revenue - total_refund
net_revenue
# Monthly Net Revenue
monthly_net_revenue = monthly_revenue.copy()
monthly_net_revenue['refund_amount'] = returns_monthly['refund_amount']
monthly_net_revenue['net_revenue'] = monthly_net_revenue['revenue'] - monthly_net_revenue['refund_amount']
monthly_net_revenue

,order_month,revenue,refund_amount,net_revenue
0,1,19618000,3616674.0,16001326.0
1,2,19104000,2771332.0,16332668.0
2,3,18658000,NaN,NaN
3,4,19160000,NaN,NaN
4,5,18070000,NaN,NaN
5,6,18550000,NaN,NaN
